To build an AI-powered medical explanation system that:

1. Extracts medical terms using NLP.
2. Retrieves medically accurate definitions using RAG.
3. Ensures source faithfulness (no hallucination).
4. Incorporates expert validation (human review).
5. Computes a final confidence score using weighted scoring.

User Input  
→ NLP Medical Term Extraction  
→ Vector Database Retrieval (RAG)  
→ Explanation Generation  
→ Source Faithfulness Check  
→ Human Validation Layer  
→ Final Confidence Score Calculation  


Final Confidence Score:

C_final = (w1 × S_retrieval)  
          + (w2 × S_source)  
          + (w3 × S_human)

Where:

- **S_retrieval** → Cosine similarity score from vector database  
- **S_source** → Faithfulness score (LLM answer vs original source)  
- **S_human** → Expert validation score (1.0 if verified)  

---


In [15]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect("medical_jargon.db")

# See available tables
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)

tables


,name
0,medical_terms
1,medical_fts
2,medical_fts_data
3,medical_fts_idx
4,medical_fts_content
5,medical_fts_docsize
6,medical_fts_config


In [16]:
df = pd.read_sql_query("SELECT * FROM medical_terms;", conn)

df.head()


,term,content,__index_level_0__,term_lower,content_length,extracted_date,summary
0,Paracetamol poisoning,"Paracetamol poisoning, also known as acetamino...",0,paracetamol poisoning,23666,2026-01-23,"Paracetamol poisoning, also known as acetamino..."
1,Acromegaly,Acromegaly is a disorder that results from exc...,1,acromegaly,21318,2026-01-23,Acromegaly is a disorder that results from exc...
2,Actinic keratosis,"Actinic keratosis (AK), sometimes called solar...",2,actinic keratosis,33330,2026-01-23,"Actinic keratosis (AK), sometimes called solar..."
3,Congenital adrenal hyperplasia,Congenital adrenal hyperplasia (CAH) is a grou...,3,congenital adrenal hyperplasia,19416,2026-01-23,Congenital adrenal hyperplasia (CAH) is a grou...
4,Adrenocortical carcinoma,Adrenocortical carcinoma (ACC) is an aggressi...,4,adrenocortical carcinoma,8252,2026-01-23,Adrenocortical carcinoma (ACC) is an aggressi...


In [17]:
pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)


,name
0,medical_terms
1,medical_fts
2,medical_fts_data
3,medical_fts_idx
4,medical_fts_content
5,medical_fts_docsize
6,medical_fts_config


In [18]:
df.columns


Index(['term', 'content', '__index_level_0__', 'term_lower', 'content_length',
       'extracted_date', 'summary'],
      dtype='str')

In [19]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("medical_jargon.db")

df = pd.read_sql_query("SELECT * FROM medical_terms;", conn)

print("Columns:")
print(df.columns)

print("\nTotal rows:", len(df))
df.head()


Columns:
Index(['term', 'content', '__index_level_0__', 'term_lower', 'content_length',
       'extracted_date', 'summary'],
      dtype='str')

Total rows: 901


,term,content,__index_level_0__,term_lower,content_length,extracted_date,summary
0,Paracetamol poisoning,"Paracetamol poisoning, also known as acetamino...",0,paracetamol poisoning,23666,2026-01-23,"Paracetamol poisoning, also known as acetamino..."
1,Acromegaly,Acromegaly is a disorder that results from exc...,1,acromegaly,21318,2026-01-23,Acromegaly is a disorder that results from exc...
2,Actinic keratosis,"Actinic keratosis (AK), sometimes called solar...",2,actinic keratosis,33330,2026-01-23,"Actinic keratosis (AK), sometimes called solar..."
3,Congenital adrenal hyperplasia,Congenital adrenal hyperplasia (CAH) is a grou...,3,congenital adrenal hyperplasia,19416,2026-01-23,Congenital adrenal hyperplasia (CAH) is a grou...
4,Adrenocortical carcinoma,Adrenocortical carcinoma (ACC) is an aggressi...,4,adrenocortical carcinoma,8252,2026-01-23,Adrenocortical carcinoma (ACC) is an aggressi...


In [20]:
# Keep only needed columns
df = df[['term', 'content', 'summary']]

# Remove nulls & duplicates
df = df.dropna().drop_duplicates()

# Clean text
df['content'] = df['content'].str.lower().str.strip()

print("Cleaned rows:", len(df))
df.head()


Cleaned rows: 901


,term,content,summary
0,Paracetamol poisoning,"paracetamol poisoning, also known as acetamino...","Paracetamol poisoning, also known as acetamino..."
1,Acromegaly,acromegaly is a disorder that results from exc...,Acromegaly is a disorder that results from exc...
2,Actinic keratosis,"actinic keratosis (ak), sometimes called solar...","Actinic keratosis (AK), sometimes called solar..."
3,Congenital adrenal hyperplasia,congenital adrenal hyperplasia (cah) is a grou...,Congenital adrenal hyperplasia (CAH) is a grou...
4,Adrenocortical carcinoma,adrenocortical carcinoma (acc) is an aggressi...,Adrenocortical carcinoma (ACC) is an aggressi...


In [21]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content=row['content'],
        metadata={
            "term": row['term'],
            "summary": row['summary']
        }
    )
    for _, row in df.iterrows()
]

print("Total documents:", len(documents))


Total documents: 901


In [22]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content=row['content'],
        metadata={
            "term": row['term'],
            "summary": row['summary']
        }
    )
    for _, row in df.iterrows()
]

print("Total documents:", len(documents))


Total documents: 901


In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

split_docs = text_splitter.split_documents(documents)

print("Total chunks:", len(split_docs))


Total chunks: 36691


In [24]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(split_docs, embedding_model)

print("Vector database created ✅")


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2515.26it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Vector database created ✅


In [14]:
conn = sqlite3.connect("medical_jargon.db")

df = pd.read_sql_query("SELECT * FROM medical_terms LIMIT 10", conn)
print("First 10 rows:")
display(df)

print("\nColumn types:")
print(df.dtypes)

print("\nMissing values:")
print(df.isna().sum())

First 10 rows:


,term,content,__index_level_0__,term_lower,content_length,extracted_date,summary
0,Paracetamol poisoning,"Paracetamol poisoning, also known as acetamino...",0,paracetamol poisoning,23666,2026-01-23,"Paracetamol poisoning, also known as acetamino..."
1,Acromegaly,Acromegaly is a disorder that results from exc...,1,acromegaly,21318,2026-01-23,Acromegaly is a disorder that results from exc...
2,Actinic keratosis,"Actinic keratosis (AK), sometimes called solar...",2,actinic keratosis,33330,2026-01-23,"Actinic keratosis (AK), sometimes called solar..."
3,Congenital adrenal hyperplasia,Congenital adrenal hyperplasia (CAH) is a grou...,3,congenital adrenal hyperplasia,19416,2026-01-23,Congenital adrenal hyperplasia (CAH) is a grou...
4,Adrenocortical carcinoma,Adrenocortical carcinoma (ACC) is an aggressi...,4,adrenocortical carcinoma,8252,2026-01-23,Adrenocortical carcinoma (ACC) is an aggressi...
5,Alcohol withdrawal syndrome,Alcohol withdrawal syndrome (AWS) is a set of ...,5,alcohol withdrawal syndrome,16646,2026-01-23,Alcohol withdrawal syndrome (AWS) is a set of ...
6,Alopecia areata,"Alopecia areata, also known as spot baldness, ...",6,alopecia areata,11883,2026-01-23,"Alopecia areata, also known as spot baldness, ..."
7,Altitude sickness,"Altitude sickness, the mildest form being acut...",7,altitude sickness,20260,2026-01-23,"Altitude sickness, the mildest form being acut..."
8,Amblyopia,"Amblyopia, also called lazy eye, is a disorder...",8,amblyopia,12923,2026-01-23,"Amblyopia, also called lazy eye, is a disorder..."
9,Amoebiasis,"Amoebiasis, or amoebic dysentery, is an infect...",9,amoebiasis,15410,2026-01-23,"Amoebiasis, or amoebic dysentery, is an infect..."



Column types:
term                   str
content                str
__index_level_0__    int64
term_lower             str
content_length       int64
extracted_date         str
summary                str
dtype: object

Missing values:
term                 0
content              0
__index_level_0__    0
term_lower           0
content_length       0
extracted_date       0
summary              0
dtype: int64


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)


In [25]:
split_docs = text_splitter.split_documents(documents)


In [26]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(split_docs, embedding_model)


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1969.70it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [29]:
results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\nResult {i+1}:")
    print("Term:", doc.metadata["term"])
    print(doc.page_content[:300])



Result 1:
Term: HELLP syndrome
de novo manifestation of hypertension with systolic pressure and diastolic pressure above 160mmhg and 110 mmhg, respectively.
proteinuria, leucocytosis and elevated uric acid concentrations > 7.8 mg.
decreased serum haptoglobin and haemoglobin levels.

Result 2:
Term: Secondary hypertension
secondary hypertension (or, less commonly, inessential hypertension) is a type of hypertension which by definition is caused by an identifiable underlying primary cause. it is much less common than the other type, called essential hypertension, affecting only 5-10% of hypertensive patients. it has m

Result 3:
Term: Secondary hypertension
hypertension secondary to endocrine disorders


In [30]:
results = vectorstore.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(results):
    print(f"\nResult {i+1}")
    print("Term:", doc.metadata["term"])
    print("Distance:", score)
    print(doc.page_content[:200])



Result 1
Term: HELLP syndrome
Distance: 0.7072136
de novo manifestation of hypertension with systolic pressure and diastolic pressure above 160mmhg and 110 mmhg, respectively.
proteinuria, leucocytosis and elevated uric acid concentrations > 7.8 mg.


Result 2
Term: Secondary hypertension
Distance: 0.71290946
secondary hypertension (or, less commonly, inessential hypertension) is a type of hypertension which by definition is caused by an identifiable underlying primary cause. it is much less common than th

Result 3
Term: Secondary hypertension
Distance: 0.7433505
hypertension secondary to endocrine disorders


In [32]:
def convert_distance_to_similarity(distance):
    return 1 / (1 + distance)

top_doc, distance = results[0]
S_retrieval = convert_distance_to_similarity(distance)

print("S_retrieval:", S_retrieval)


S_retrieval: 0.58574975


In [33]:
vectorstore.save_local("faiss_index")
